# Exercise 3 - Data Lake on S3

In [7]:
from pyspark.sql import SparkSession
import os
import configparser

# Make sure that your AWS credentials are loaded as env vars

In [9]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read('dl.cfg')

os.environ["AWS_ACCESS_KEY_ID"]= config['AWSUSER']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWSUSER']['AWS_SECRET_ACCESS_KEY']

# Create spark session with hadoop-aws package

In [10]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

## The etl.py script runs without errors.

The script, etl.py, runs in the terminal without errors. The script reads song_data and load_data from S3, transforms them to create five different tables, and writes them to partitioned parquet files in table directories on S3.

## Analytics tables are correctly organized on S3.

Each of the five tables are written to parquet files in a separate analytics directory on S3. Each table has its own folder within the directory. Songs table files are partitioned by year and then artist. Time table files are partitioned by year and month. Songplays table files are partitioned by year and month.

## The correct data is included in all tables.

Each table includes the right columns and data types. Duplicates are addressed where appropriate.

# Load data from S3

In [11]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://spark-project4/"
    

In [16]:
df = spark.read.json('s3a://udacity-dend/song_data/*/*/*.json') # s3a://udacity-dend/song_data/*/*/*/*.json to read the whole data

Py4JJavaError: An error occurred while calling o199.json.
: com.amazonaws.services.s3.model.AmazonS3Exception: Status Code: 403, AWS Service: Amazon S3, AWS Request ID: CE8D752C065CC226, AWS Error Code: null, AWS Error Message: Forbidden, S3 Extended Request ID: gWjkFIMTsi7dsDrwkrlhSbGOWGzBL41aj1VeBZO0AbP2y1L7O3QSE30Om3WM2zBb2LzTYuj1TJ4=
	at com.amazonaws.http.AmazonHttpClient.handleErrorResponse(AmazonHttpClient.java:798)
	at com.amazonaws.http.AmazonHttpClient.executeHelper(AmazonHttpClient.java:421)
	at com.amazonaws.http.AmazonHttpClient.execute(AmazonHttpClient.java:232)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3528)
	at com.amazonaws.services.s3.AmazonS3Client.getObjectMetadata(AmazonS3Client.java:976)
	at com.amazonaws.services.s3.AmazonS3Client.getObjectMetadata(AmazonS3Client.java:956)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:892)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.listStatus(S3AFileSystem.java:734)
	at 

In [9]:
df.printSchema()
df.show(5)

NameError: name 'df' is not defined

# schema

In [7]:

from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Dat, TimestampType


songSchema = R([
        Fld("artist_id",Str()),
        Fld("artist_latitude",Dbl()),
        Fld("artist_location",Str()),
        Fld("artist_longitude",Dbl()),
        Fld("artist_name",Str()),
        Fld("duration",Dbl()),
        Fld("num_songs",Int()),
        Fld("title",Str()),
        Fld("year",Int()),
    ])

df = spark.read.json("s3a://udacity-dend/song_data/A/B/C/*.json",
                     schema= songSchema)

In [8]:
df.printSchema()
df.limit(5).toPandas()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)



artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARLTWXK1187FB5A3F8         32.74863   Fort Worth, TX         -97.32925   
1  ARIOZCU1187FB3A3DC              NaN       Hamlet, NC               NaN   
2  ARPFHN61187FB575F6         41.88415      Chicago, IL         -87.63241   
3  AR5S9OB1187B9931E3         34.05349  Los Angeles, CA        -118.24532   
4  AR5T40Y1187B9996C6              NaN    Lulea, Sweden               NaN   

        artist_name   duration  num_songs  \
0       King Curtis  326.00771          1   
1     JOHN COLTRANE  220.44689          1   
2       Lupe Fiasco  279.97995          1   
3       Bullet Boys  156.62975          1   
4  The Bear Quartet  249.31220          1   

                                               title  year  
0      A Whiter Shade Of Pale (Live @ Fillmore West)     0  
1  Giant Steps (Alternate Version_ Take 5_ Altern...     0  
2           Streets On Fire (Explicit Album Version)     0  
3                         All Day & All Of The Night     0  
4                        I Remember Nights Wide Open  1998

# saving the data

In [9]:
import  pyspark.sql.functions as F


song_fields = ["title", "artist_id","year", "duration"]

songs_table = (df
               .select(song_fields)
               .dropDuplicates()
               .withColumn("song_id", monotonically_increasing_id())
              )



In [14]:
songs_table.limit(5).toPandas()

title           artist_id  year   duration       song_id
0            Intro  AR558FS1187FB45658  2003   75.67628   51539607552
1  I Need A Mother  ARZGTK71187B9AC7F5  2010  158.01424  137438953472
2            Intro  ARAADXM1187FB3ECDB  1999   67.63057  335007449088
3              XXX  ARZJDBC1187FB52056  1984  327.00036  395136991232
4     Race Of Hate  AREH0O41187FB4C405  1997  311.90159  481036337152

In [13]:
songs_table.count()

23

### Apache Parquet Introduction

Apache Parquet is a columnar file format that provides optimizations to speed up queries and is a far more efficient file format than CSV or JSON, supported by many data processing systems.




### Spark Write DataFrame to Parquet file format

Using spark.write.parquet() function we can write Spark DataFrame to Parquet file, and parquet() function is provided in DataFrameWriter class. 




### Spark parquet partition – Improving performance

> Partitioning is a feature of many databases and data processing frameworks and it is key to make jobs work at scale. We can do a parquet file partition using spark partitionBy function.

> Parquet Partition creates a folder hierarchy for each spark partition; we have mentioned the first partition as year followed by artist_id hence, it creates a year folder inside the artist_id folder.

In [ ]:
    
(songs_table
 .write
 .partitionBy("year", "artist_id")
 .parquet(output_data + 'songs/')
)

In [ ]:
artists_fields = ["artist_id",
                  "artist_name as name",
                  "artist_location as location",
                  "artist_latitude as latitude",
                  "artist_longitude as longitude"]

artists_table = df.selectExpr(artists_fields).dropDuplicates()

artists_table.write.parquet(output_data + 'artists/')